In [ ]:
# Installing xgboost for our notebook instance
!pip3 install xgboost==0.82

In [ ]:
#importing necessary libraries
import pandas as pd
import xgboost as xgb
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery

In [ ]:
# BigQuery has made many datasets publicly available for your exploration. For this lab, we

In [ ]:
# We have to specify the top 100 
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""

In [ ]:
df = bigquery.Client().query(query).to_dataframe()
df.head()

In [ ]:
df['is_male'].value_counts()

In [ ]:
df.isnull().sum() #checking null values

In [ ]:
df = df.dropna(axis=0) #dropping all null values

In [ ]:
df.isnull().sum() #checking the null values after dropping the null values

In [ ]:
df.head() #top most rows

In [ ]:
# Shuffle the data
df = shuffle(df,random_state = 42)

In [ ]:
df.head()

In [ ]:
#--------------------------

In [ ]:
# query1="""
# SELECT
#   count(plurality),
# FROM
#   publicdata.samples.natality
#   WHERE year >2000
# """

In [ ]:
# df = bigquery.Client().query(query1).to_dataframe()
# df.head()

In [ ]:
# query2="""
# SELECT
#   count(plurality),
# FROM
#   publicdata.samples.natality
#   WHERE year = 2000
# """

In [ ]:
# df = bigquery.Client().query(query2).to_dataframe()

In [ ]:
# df

In [ ]:
#----------------------

In [ ]:
labels = df['weight_pounds'] #defining the labels
data = df.drop(columns = ['weight_pounds'],axis = 1) #dropping the column

In [ ]:
data.head()

In [ ]:
#changing the datatype
data['is_male'] = data['is_male'].astype(float)
data['mother_age'] = data['mother_age'].astype(float)
data['plurality'] = data['plurality'].astype(float)
data['gestation_weeks'] = data['gestation_weeks'].astype(float)

In [ ]:
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
#splitting the dataset
x,y =data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [ ]:
model = xgb.XGBRegressor(objective = 'reg:linear') #Making the model

In [ ]:
model.fit(x_train,y_train) #fitting the model

In [ ]:
y_pred = model.predict(x_test) #predicting the value for test test

In [ ]:
#finding the loss
from sklearn.metrics import r2_score
r2_score(y_test,pred)

In [ ]:
for i in range(20):
    print('Predicted weight: ', y_pred[i])
    print('Actual weight: ', y_test.iloc[i])
    print()

In [ ]:
model.save_model('sachin.bst')  #saving the model

In [ ]:
# # Deployment
# Step 1: Create a Cloud Storage bucket for our model.

In [ ]:
# Update these to your own GCP project, model, and version names
GCP_PROJECT = 'My first Project'
MODEL_BUCKET = 'gs://sachin-buckets'
VERSION_NAME = 'v1'
MODEL_NAME = 'sachin-model'

In [ ]:
!gsutil mb $MODEL_BUCKET

In [ ]:
!gsutil cp ./sachin_model.bst $MODEL_BUCKET